In [62]:
# Импорт библиотек
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [72]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [63]:
# Чтение датасета
df = pd.read_csv('../data/_data.csv')

In [64]:
# удаляем столбец `Unnamed: 0`
df = df.iloc[:, 1:]

# переименовываем столбцы
NEW_COLUMNS = {
    'ID  объявления':           'id',
    'Количество комнат':        'rooms',
    'Тип':                      'type',
    'Метро':                    'metro_station',
    'Адрес':                    'address',
    'Площадь, м2':              'area',
    'Дом':                      'building_type',
    'Парковка':                 'parking',
    'Цена':                     'price',
    'Телефоны':                 'phones',
    'Описание':                 'description',
    'Ремонт':                   'renovation',
    'Площадь комнат, м2':       'rooms_area',
    'Балкон':                   'balcony',
    'Окна':                     'windows',
    'Санузел':                  'bathroom',
    'Можно с детьми/животными': 'kids_pets',
    'Дополнительно':            'extra',
    'Название ЖК':              'complex_title',
    'Серия дома':               'building_series',
    'Высота потолков, м':       'ceiling_height',
    'Лифт':                     'elevator',
    'Мусоропровод':             'trash_chute',
    'Ссылка на объявление':     'listing_url',
}
df = df.rename(columns=NEW_COLUMNS)

In [65]:
# 1. Создаем столбец с главной ценой
# r'([\d\.]+)' Это регулярное выражение, которое используется для поиска чисел с точкой. 
# \d — означает любую цифру от 0 до 9. 
# [\d\.]+ — это: «одна или больше цифр и/или точек подряд». 
df['main_price'] = df['price'].str.extract(r'([\d\.]+)').astype(float)

# 2. Поработаем с залогом
# Ищем слово Залог, дефис, и число после него
# *-\s* это мы так страхуемся, чтобы захватились пробелы до и после дефиса"""
df['deposit'] = df['price'].str.extract(r'Залог\s*-\s*([\d\s]+)')[0]
df['deposit'] = df['deposit'].str.replace(' ', '').astype(float)

# 3. Коммунальные услуги - выводим пока что только булевы значения True/False
df['utilities_included'] = df['price'].str.contains('Коммунальные услуги включены', case=False, na=False)

# 4. Срок аренды
# Ищем слово Срок аренды, дефис, и слова после него
df['rental_term'] = df['price'].str.extract(r'Срок аренды\s*-\s*([а-яА-Яa-zA-Z ]+)')

# 5. Предоплата
# Ищем слово Предоплата, дефис, и слова после него
df['prepayment_months'] = df['price'].str.extract(r'Предоплата\s*(\d+)')
df['prepayment_months'] = df['prepayment_months'].astype(float)

df.head(2)

,id,rooms,type,metro_station,address,area,building_type,parking,price,phones,description,renovation,rooms_area,balcony,windows,bathroom,kids_pets,extra,complex_title,building_series,ceiling_height,elevator,trash_chute,listing_url,main_price,deposit,utilities_included,rental_term,prepayment_months
0,271271157,4,Квартира,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79166369231,Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157,500000.0,500000.0,True,Длительный,1.0
1,271634126,4,Квартира,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79850608590,Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.5,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126,500000.0,500000.0,True,Длительный,1.0


# DMITRY

### Create aditional columns from "rooms"

In [201]:
df['num_rooms'] = df['rooms'].str.extract(r'(\d+)').astype('Int64')
df['num_rooms'].unique()

<IntegerArray>
[4, 2, 3, 5, 6, 1, <NA>]
Length: 7, dtype: Int64

In [67]:
df['type_rooms'] = df['rooms'].str.extract(r'(\s\D+)')
df['type_rooms'].unique()

array([nan, ' Оба варианта', ' Изолированная', ' Смежная'], dtype=object)

### Create aditional columns from "building_type"

In [ ]:
df['floor'] = df['building_type'].str.extract(r'(\d{1,3})/').astype('Int64')
df['floor'].unique()

array([ 5, 12,  3,  4,  2,  6, 19,  8, 10, 21,  7,  9, 26, 25, 11, 24, 23,
        1, 13, 14, 16, 15, 17, 22, 18, 20, 27, 30, 28, 33, 35, 39, 31, 36,
       34, 29, 46, 38, 54, 61, 52, 40, 47, 43, 58, 55, 50, 57, 37, 41, 53,
       32, 75, 45, 48, 85, 71, 70, 49, 59, 65, 42])

In [ ]:
df['total_floors'] = df['building_type'].str.extract(r'/(\d{1,3})').astype('Int64')
df['total_floors'].unique()

array([ 16,   6,  26,   7,   5,   4,  10,   8,   9,   3,  13,  17,  12,
        14,  11,  24,  15,  22,  19,  18,   2,  23,  20,  21,  25,  39,
        45,  33,  28,  27,  30,  29,  38,  37,   1,  58,  35,  31,  34,
        32,  53,  54,  50,  46,  52,  44,  43,  36,  55,  47,  40,  65,
        77,  78,  80,  75,  68,  85,  86,  41,  48,  49,  60,  59,  72,
        42,  73,  95,  51,  76,  62,  57, 116])

### Create aditional columns from "parking"

In [80]:
df['parking'].fillna('нет', inplace=True)
df['parking_categories'] = pd.factorize(df['parking'])[0]
df['parking_categories'].unique()

array([0, 1, 2, 3, 4, 5])

### Create aditional columns from "kids_pets"

In [86]:
df['kids_allowed'] = np.where(df['kids_pets'].str.contains('детьми'), 1, 0)
df['kids_allowed'].unique()

array([1, 0])

In [89]:
df['pets_allowed'] = np.where(df['kids_pets'].str.contains('животными'), 1, 0)
df['pets_allowed'].unique()

array([1, 0])

### Create aditional columns from "elevator"

In [199]:
df['elevator_num'] = df['elevator'].str.extract(r'(\d{1,2})\).*(\d{1,2})\).*|(\d{1,2})\)').astype('Int64').sum(axis=1, min_count=1)
df['elevator_num'].unique()

<IntegerArray>
[5, 2, 1, <NA>, 3, 4, 6, 8, 7, 60, 16, 50]
Length: 12, dtype: Int64